In [1]:
import keras
from keras.models import Sequential
from keras.layers import MaxPooling2D, Dense, Dropout, Flatten, Conv2D, BatchNormalization, Activation
from keras import optimizers
import os


C:\Users\I353565\AppData\Local\Anaconda\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
batch_size = 32
num_classes = 10
epochs = 50
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'
input_size = (3,227,227)

In [3]:
from keras.preprocessing.image import ImageDataGenerator

#### Loading Data

In [4]:
# This will do preprocessing and realtime data augmentation:
# Compute quantities required for feature-wise normalization
# (std, mean, and principal components if ZCA whitening is applied).
train_datagen = ImageDataGenerator(
    width_shift_range=0.1,
    # randomly shift images vertically (fraction of total height)
    height_shift_range=0.1,
    fill_mode='nearest',
    horizontal_flip=True,  # randomly flip images ;Mentioned in the paper
)

test_datagen = ImageDataGenerator()

train_gen = train_datagen.flow_from_directory(
            '../Data/Train',
            batch_size=batch_size,
            shuffle=True,
            target_size=input_size[1:],
            class_mode='categorical')

validation_gen = test_datagen.flow_from_directory(
                '../Data/Test',
                batch_size=batch_size,
                shuffle=True,
                target_size=input_size[1:],
                class_mode='categorical')





Found 2002 images belonging to 2 classes.
Found 800 images belonging to 2 classes.


In [7]:
#Note that I'll be using Batch Normalization instead of LRN
def create_alexnet():
    inputShape = (227,227,3)   # Note that input shape is mentioned as 224x224x3 in AlexNet Paper. 
                               # It actually is 227X227X3 as mentioned by Andrej Karpathy in CS231N
    
    alexnet = Sequential()
    
    # Layer 1.
        # [Input] ==> 227x227x3
        # --> 227x227x3 ==> [Convolution: size=(11x11x3)x96, strides=4, padding=valid] ==> 55x55x96
        # --> 55x55x96 ==> [ReLU] ==> 55x55x96
        # --> 55x55x96 ==> [Batch Normalization] ==> 55x55x96
        # --> 55x55x96 ==> [Max-Pool: size=3x3, strides=2, padding=valid] ==> 27x27x96
        # --> [Output] ==> 27x27x96
    alexnet.add(Conv2D(filters = 96, kernel_size = (11,11), strides = (4,4), input_shape = (227,227,3), padding = 'valid'))
    alexnet.add(Activation('relu'))
    alexnet.add(BatchNormalization(axis = 1))
    alexnet.add(MaxPooling2D(pool_size = (3,3), strides = 2, padding = 'valid'))
    
    # Layer 2.
        # [Input] ==> 27x27x96
        # --> 27x27x96 ==> [Convolution: size=(5x5x96)x256, strides=1, padding=same] ==> 27x27x256
        # --> 27x27x256 ==> [ReLU] ==> 27x27x256
        # --> 27x27x256 ==> [Batch Normalization] ==> 27x27x256
        # --> 27x27x256 ==> [Max-Pool: size=3x3, strides=2, padding=valid] ==> 13x13x256
        # --> [Output] ==> 13x13x256
    alexnet.add(Conv2D(filters = 256, kernel_size = (5,5), strides = (1,1), padding= 'same'))
    alexnet.add(Activation('relu'))
    alexnet.add(BatchNormalization(axis = 1))
    alexnet.add(MaxPooling2D(pool_size = (3,3), strides = 2, padding = 'valid'))
    
    # Layer 3.
        # [Input] ==> 13x13x256
        # --> 13x13x256 ==> [Convolution: size=(3x3x256)x384, strides=1, padding=same] ==> 13x13x384
        # --> 13x13x384 ==> [ReLU] ==> 13x13x384
        # --> [Output] ==> 13x13x384
    alexnet.add(Conv2D(filters = 384, kernel_size = (3,3), strides = (1,1), padding='same'))
    alexnet.add(Activation('relu'))
    
    # Layer 4.
        # [Input] ==> 13x13x384
        # --> 13x13x384 ==> [Convolution: size=(3x3x384)x384, strides=1, padding=same] ==> 13x13x384
        # --> 13x13x384 ==> [ReLU] ==> 13x13x384
        # --> [Output] ==> 13x13x384
    alexnet.add(Conv2D(filters = 384, kernel_size = (3,3), strides = (1,1), padding='same'))
    alexnet.add(Activation('relu'))
    
    # Layer 5.
        # [Input] ==> 13x13x384
        # --> 13x13x384 ==> [Convolution: size=(3x3x384)x256, strides=1, padding=same] ==> 13x13x256
        # --> 13x13x256 ==> [ReLU] ==> 13x13x256
        # --> 13x13x256 ==> [Max-Pool: size=3x3, strides=2, padding=valid] ==> 6x6x256
        # --> [Output] ==> 6x6x256
    alexnet.add(Conv2D(filters = 256, kernel_size = (3,3), strides = (1,1), padding = 'same'))
    alexnet.add(Activation('relu'))
    alexnet.add(MaxPooling2D(pool_size = (3,3), strides = 2, padding = 'valid'))
    
    # Layer 6.
        # [Input] ==> 6x6x256=9216
        # --> 9216 ==> [Fully Connected: neurons=4096] ==> 4096
        # --> 4096 ==> [ReLU] ==> 4096
        # --> [Output] ==> 4096
    alexnet.add(Flatten())
    alexnet.add(Dense(4096))
    alexnet.add(Activation("relu"))
   
    # Layer 7.
        # [Input] ==> 4096
        # --> 4096 ==> [Fully Connected: neurons=4096] ==> 4096
        # --> 4096 ==> [ReLU] ==> 4096
        # --> [Output] ==> 4096
    
    alexnet.add(Dense(4096))
    alexnet.add(Activation("relu"))
    
    # Layer 8.
        # [Input] ==> 4096
        # --> 4096 ==> [Logits: neurons=1000] ==> 1000
        # --> [Output] ==> 1000
    alexnet.add(Dense(2))
    alexnet.add(Activation("softmax"))
    
    alexnet.summary()
    return alexnet
    

In [6]:
model = create_alexnet()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 55, 55, 96)        34944     
_________________________________________________________________
activation_1 (Activation)    (None, 55, 55, 96)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 55, 55, 96)        220       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 27, 27, 96)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 27, 27, 256)       614656    
_________________________________________________________________
activation_2 (Activation)    (None, 27, 27, 256)       0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 27, 27, 256)       108       
__________

In [ ]:
sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='mse',
              optimizer=sgd,
              metrics=['accuracy'])

In [ ]:
# Fit the model on the batches generated by train_datagen
history = model.fit_generator(train_gen,
                              steps_per_epoch=100,
                              epochs=epochs,
                              validation_steps=20,
                              validation_data=validation_gen, 
                              verbose = 1)

In [ ]:
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])